# ライブラリのインポート / データの読み込み

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd

pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

%matplotlib inline

from matplotlib import pyplot as plt
import matplotlib.ticker as mtick # For specifying the axes tick format 

import seaborn as sns
import re

import json, os, gc, math, time
import datetime
import collections
from tqdm import tqdm
import glob

from statistics import mean
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold

from sklearn import metrics

import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

In [3]:
#ドライブへのアクセス
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/00_datascience/19_ufj_bank"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/00_datascience/19_ufj_bank


In [4]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
sub = pd.read_csv('input/sample_submit.csv')

In [5]:
train.head()

,id,goal,country,duration,category1,category2,html_content,state
0,train_00000,20001-21000,US,45,art,mixed media,"<div class=""contents""><div><p><a href=""http://dummy.com"">http://dummy.com<p>In its first year, The Shillito's Elves Display won an international \ndesign award for Shillito's department store. The elves display is arts\n and crafts at its finest. The mixed media exhibit displays the talents\n of local fine arts graduates, and the display, while ""folksy"", is as \ntechnologically advanced as Disney World's famous ""It's a Small World"" \nride. </p><p>The Shillito's Elves attracted close to 100...",1
1,train_00001,19001-20000,US,59,food,restaurants,"<div class=""contents""><div><p>Cultural Pretzel Sports Bar is a place where people can come and watch their favorite local and world wide sports teams, while enjoying their favorite ethnic foods on a soft pretzel. Our menu includes a variety of appetizers and soft pretzels. Our pretzels include - Mexican Taco, Mexican Steak Fajita, Greek Gyro, Italian Sausage and Peppers, and American Steak Philly Soft Pretzels. With more to be added to the menu as the business grows.</p><p>Cultural Pretze...",0
2,train_00002,2001-3000,US,38,art,performance art,"<div class=""contents""><div><p>I want to perform this piece guerilla style, off the back off a 24 Ft truck on Christmas Eve, amid last minute Christmas shopping in New York City.</p>\n<p>Our truck pulls in, the door rolls up and I perform. </p>\n<p>New York is a city where you just do it - no excuses. A place where the new becomes the normal real fast and where the artist is challenged to capture the attention and the hearts of the unshockable and unstoppable. In times of celebration and re...",0
3,train_00003,1001-2000,US,30,art,mixed media,"<div class=""contents""><div><div class=""template asset"" contenteditable=""false"" data-alt-text="""" data-caption="""" data-id=""_xxx_"">\n<figure>\n<img alt="""" class=""fit lazyload"" data-src=""http://dummy.com""/>\n</figure>\n</div>\n<h1 class=""page-anchor"" id=""_xxx_"">\n<figure>\n<img alt=""Canyon de Chelley, Dine' (Navajo) Reservation, Arizona. Photo by: Demian Dine' Yazhi'"" class=""fit lazyload"" data-src=""http://dummy.com""/>\n<figcaption class=""px2"">Canyon de Chelley, Dine' (Navajo) Reservation, Arizo...",1
4,train_00004,1001-2000,US,29,film & video,webseries,"<div class=""contents""><div><p>The story of the show, both on and off screen, is about daring to dream after something impossible.Even with an incredible amount of difficulty and opposition, the project continued to grow in size and depth. A year ago, we were finally able to release the pilot episode of Azusa Ghost Hunter's Society (check it out! <a href=""http://dummy.com"">\n<figure>\n<img alt="""" class=""fit lazyload"" data-src=""http://dummy.com""/>\n</figure>\n</div>\n<p>Which brings us to now...",1


# bertによる特徴抽出

In [6]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

In [7]:
for i in range(len(train)):
  train.loc[i, 'cleaned_text'] = remove_html(train.loc[i, 'html_content'])

for i in range(len(test)):
  test.loc[i, 'cleaned_text'] = remove_html(test.loc[i, 'html_content'])

In [8]:
#すでに抽出済みの特徴量を使用
text_train_df = pd.read_csv('input/05_seq_train_df.csv')
text_train_df.drop(['Unnamed: 0'], axis=1, inplace=True)
text_test_df = pd.read_csv('input/05_seq_test_df.csv')
text_test_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [9]:
merge_text = pd.concat([text_train_df, text_test_df], ignore_index=True)

# ラベルデータの前処理

In [10]:
# ワンホットエンコーディング
merge_df = pd.concat([train, test], ignore_index=True)
dummy_cols = ['goal',	'country', 'category1',	'category2']
from sklearn.preprocessing import OneHotEncoder

dummy_df = pd.get_dummies(merge_df[dummy_cols])

In [11]:
merge_df_new = pd.merge(merge_df, dummy_df, left_index=True, right_index=True)

In [12]:
#ラベルエンコーディング
from sklearn.preprocessing import LabelEncoder

le_cols = ['goal', 'country', 'category1', 'category2']

#カテゴリ変数をループしてlabel encoding
for c in le_cols:
  le = LabelEncoder()
  le.fit(merge_df_new[c])
  merge_df_new[c] = le.transform(merge_df_new[c])

# tfidfによる特徴抽出

In [13]:
#tfidfによる特徴抽出
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer()
features = tv.fit_transform(merge_df_new["cleaned_text"].fillna(""))

In [14]:
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
from sklearn.pipeline import Pipeline



tfidf_svd = Pipeline(steps=[
    ("TfidfVectorizer", TfidfVectorizer()),
    ("TruncatedSVD", TruncatedSVD(n_components=50, random_state=42))
])

features_svd = tfidf_svd.fit_transform(merge_df_new["cleaned_text"].fillna(""))

In [15]:
svd_df_merge = pd.DataFrame(features_svd)
svd_df_merge.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,0.294975,0.000612,0.023093,-0.043301,-0.004649,0.020541,-0.045337,0.023749,-0.013550,0.034197,-0.013022,0.006791,-0.034909,0.006838,-0.015130,0.005652,-0.006562,0.028293,0.020380,-0.021149,0.014203,0.012766,-0.021624,0.008557,-0.033513,0.021839,-0.029243,0.018258,0.040335,0.013609,0.009730,-0.025807,-0.011607,-0.016283,-0.001402,-0.008286,0.003465,-0.027083,-0.020888,-0.006050,0.025245,0.050684,0.018035,-0.003444,-0.016337,-0.011634,-0.045625,-0.018838,-0.031891,-0.005867
1,0.163203,-0.003916,0.027252,0.018163,-0.002540,-0.027746,0.007868,0.058626,-0.076375,0.005537,0.001517,0.063067,0.065023,-0.036968,0.008155,-0.003813,0.084616,-0.047082,-0.007784,-0.024780,-0.022541,0.037703,0.000743,-0.016317,0.020467,-0.004459,0.013540,-0.020566,-0.011280,-0.014888,-0.022607,0.005381,0.006592,0.031154,-0.001644,-0.021823,0.001153,-0.012336,0.011720,0.024251,0.008138,-0.020235,0.012740,0.010774,0.020248,-0.006551,-0.008633,-0.002234,0.000846,-0.026562
2,0.416711,0.004678,-0.052108,-0.079082,-0.010638,0.082929,-0.018381,0.036628,-0.071629,-0.046454,-0.014980,0.041316,-0.020198,-0.010975,-0.087276,-0.007349,-0.030171,0.014232,-0.013043,-0.046913,-0.023943,0.036428,0.027846,-0.033865,-0.007355,-0.006390,0.046919,-0.010758,0.026100,-0.038122,-0.011840,0.023103,0.024219,-0.019092,0.007193,-0.044791,-0.015232,-0.034269,0.004956,0.011575,-0.000471,0.000399,0.030256,-0.018961,-0.024183,0.009083,0.049905,-0.024426,-0.004114,-0.069994
3,0.354831,0.040357,-0.074435,-0.139759,-0.016492,0.005695,-0.074024,-0.054700,-0.033431,0.030271,0.012615,-0.062605,0.008202,-0.000162,-0.032588,-0.007491,-0.049390,0.035334,-0.013804,-0.028062,0.019617,-0.020532,0.060234,-0.026357,-0.016961,-0.027071,0.033834,-0.005823,-0.077913,0.026541,0.035371,-0.021575,0.007319,0.010192,-0.005169,0.009147,0.007499,0.017499,0.003257,0.016648,-0.031395,-0.018403,0.033881,0.000654,-0.004903,-0.000448,0.007058,-0.027521,0.001671,-0.018905
4,0.260418,0.001437,0.023826,-0.039866,-0.012193,0.048334,0.003215,0.018024,0.000459,0.010460,-0.007537,0.005912,-0.013804,-0.001257,0.006475,-0.006275,-0.013372,-0.004754,-0.011707,0.004615,-0.020253,0.022072,-0.038320,-0.031949,0.021487,-0.045155,0.025808,0.028343,-0.050307,-0.025407,-0.003430,0.009133,0.003049,-0.006112,0.014727,-0.010559,-0.001788,-0.021023,-0.015821,-0.001193,-0.018419,0.000753,0.000358,0.005429,0.016303,0.036587,0.004850,-0.007258,0.013238,-0.015719


# データのマージ/kmeans用の前処理

In [16]:
#不要なカラムの定義
non_use_cols = ['id',	'html_content',	'cleaned_text', 'state']

In [17]:
#bertから抽出した特徴量を追加
full_merge_df = pd.merge(merge_df_new.drop(non_use_cols, axis=1), merge_text, left_index=True, right_index=True)

In [18]:
#tfidfから抽出した特徴量を追加
full_merge_df_fin = pd.merge(full_merge_df, svd_df_merge, left_index=True, right_index=True)

In [19]:
full_merge_df_fin.head()

,goal,country,duration,category1,category2,goal_1-1000,goal_100000+,goal_10001-11000,goal_1001-2000,goal_11001-12000,goal_12001-13000,goal_13001-14000,goal_14001-15000,goal_15001-16000,goal_16001-17000,goal_17001-18000,goal_18001-19000,goal_19001-20000,goal_20001-21000,goal_2001-3000,goal_21001-22000,goal_22001-23000,goal_23001-24000,goal_24001-25000,goal_25001-26000,goal_26001-27000,goal_27001-28000,goal_28001-29000,goal_29001-30000,goal_30001-31000,goal_3001-4000,goal_31001-32000,goal_32001-33000,goal_33001-34000,goal_34001-35000,goal_35001-36000,goal_36001-37000,goal_37001-38000,goal_38001-39000,goal_39001-40000,goal_40001-41000,goal_4001-5000,goal_41001-42000,goal_42001-43000,goal_43001-44000,goal_44001-45000,goal_45001-46000,goal_46001-47000,goal_47001-48000,goal_48001-49000,goal_49001-50000,goal_50001-51000,goal_5001-6000,goal_51001-52000,goal_52001-53000,goal_53001-54000,goal_54001-55000,goal_55001-56000,goal_56001-57000,goal_57001-58000,goal_58001-59000,goal_59001-60000,goal_60001-61000,goal_6001-7000,goal_61001-62000,goal_62001-63000,goal_63001-64000,goal_64001-65000,goal_65001-66000,goal_66001-67000,goal_67001-68000,goal_68001-69000,goal_69001-70000,goal_70001-71000,goal_7001-8000,goal_71001-72000,goal_72001-73000,goal_73001-74000,goal_74001-75000,goal_75001-76000,goal_76001-77000,goal_77001-78000,goal_78001-79000,goal_79001-80000,goal_80001-81000,goal_8001-9000,goal_81001-82000,goal_82001-83000,goal_83001-84000,goal_84001-85000,goal_85001-86000,goal_86001-87000,goal_87001-88000,goal_88001-89000,goal_89001-90000,goal_90001-91000,goal_9001-10000,goal_91001-92000,goal_92001-93000,goal_93001-94000,goal_94001-95000,goal_95001-96000,goal_96001-97000,goal_97001-98000,goal_98001-99000,goal_99001-100000,country_AT,country_AU,country_BE,country_CA,country_CH,country_DE,country_DK,country_ES,country_FR,country_GB,country_HK,country_IE,country_IT,country_JP,country_LU,country_MX,country_NL,country_NO,country_NZ,country_SE,country_SG,country_US,category1_art,category1_comics,category1_crafts,category1_dance,category1_design,category1_fashion,category1_film & video,category1_food,category1_games,category1_journalism,category1_music,category1_photography,category1_publishing,category1_technology,category1_theater,category2_3d printing,category2_academic,category2_accessories,category2_action,category2_animals,category2_animation,category2_anthologies,category2_apparel,category2_apps,category2_architecture,category2_art books,category2_audio,category2_bacon,category2_blues,category2_calendars,category2_camera equipment,category2_candles,category2_ceramics,category2_children's books,category2_childrenswear,category2_chiptune,category2_civic design,category2_classical music,category2_comedy,category2_comic books,category2_community gardens,category2_conceptual art,category2_cookbooks,category2_country & folk,category2_couture,category2_crochet,category2_digital art,category2_diy,category2_diy electronics,category2_documentary,category2_drama,category2_drinks,category2_electronic music,category2_embroidery,category2_events,category2_experimental,category2_fabrication tools,category2_faith,category2_family,category2_fantasy,category2_farmer's markets,category2_farms,category2_festivals,category2_fiction,category2_fine art,category2_flight,category2_food trucks,category2_footwear,category2_gadgets,category2_gaming hardware,category2_glass,category2_graphic design,category2_graphic novels,category2_hardware,category2_hip-hop,category2_horror,category2_illustration,category2_immersive,category2_indie rock,category2_installations,category2_interactive design,category2_jazz,category2_jewelry,category2_kids,category2_knitting,category2_latin,category2_letterpress,category2_literary journals,category2_literary spaces,category2_live games,category2_makerspaces,category2_metal,category2_mixed media,category2_mobile games,category2_movie theaters,category2_music videos,category2_musical,category2_narrative film,category2_nature,category2_nonfict

In [20]:
#durationのスケーリング
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
full_merge_df_fin['duration'] = std.fit_transform(pd.DataFrame(full_merge_df_fin['duration']))

# kmeansによるクラスタリング

In [23]:
from sklearn.cluster import KMeans
non_use_cols_for_km = ['goal', 'country', 'category1', 'category2']

In [ ]:
# エルボー方による推定。クラスター数を1から20に増やして、それぞれの距離の総和を求める
dist_list =[]
for i in range(1,20):
    print('今はクラスター数{}のクラスタリングを実行しています。'.format(i))
    kmeans= KMeans(n_clusters=i, init='random', random_state=0)
    kmeans.fit(full_merge_df_fin.drop(non_use_cols_for_km, axis=1))
    dist_list.append(kmeans.inertia_)
    
# グラフを表示
plt.plot(range(1,20), dist_list,marker='+')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')

In [24]:
# n_clusters=5, 10, 15, 20で実行

kmeans = KMeans(init='random', n_clusters=5, random_state=0)
kmeans.fit(full_merge_df_fin.drop(non_use_cols_for_km, axis=1))
km_label_df5 =pd.Series(kmeans.labels_, name='cluster_number_{}'.format(5))

kmeans = KMeans(init='random', n_clusters=10, random_state=0)
kmeans.fit(full_merge_df_fin.drop(non_use_cols_for_km, axis=1))
km_label_df10 =pd.Series(kmeans.labels_, name='cluster_number_{}'.format(10))

kmeans = KMeans(init='random', n_clusters=15, random_state=0)
kmeans.fit(full_merge_df_fin.drop(non_use_cols_for_km, axis=1))
km_label_df15 =pd.Series(kmeans.labels_, name='cluster_number_{}'.format(15))

kmeans = KMeans(init='random', n_clusters=20, random_state=0)
kmeans.fit(full_merge_df_fin.drop(non_use_cols_for_km, axis=1))
km_label_df20 =pd.Series(kmeans.labels_, name='cluster_number_{}'.format(20))

In [25]:
#マージ
full_merge_df_fin['cluster_number_5'] = km_label_df5
full_merge_df_fin['cluster_number_10'] = km_label_df10
full_merge_df_fin['cluster_number_15'] = km_label_df15
full_merge_df_fin['cluster_number_20'] = km_label_df20

# aggrigation特徴量の作成

In [30]:
#aggrigationの際にgroup化するカラム
group_cols = ['goal',	'country', 'category1',	'category2', 'cluster_number_5', 'cluster_number_10', 'cluster_number_15', 'cluster_number_20']

In [37]:
#関数定義
def agg_func_duration(input_df, cols):
  agg_func = ["mean", "median", "max", "min", "std"]
  tmp = input_df.groupby([c])["duration"].agg(agg_func)
  output_df = pd.merge(input_df, tmp, how="left", on=[c])[agg_func].add_suffix("_agg_func_duration")

  return output_df

def agg_func_state(input_df, cols):
  agg_func = ["mean", "median", "max", "min", "std"]
  tmp = input_df.groupby([c])["duration"].agg(agg_func)
  output_df = pd.merge(input_df, tmp, how="left", on=[c])[agg_func].add_suffix("_agg_func_state")

  return output_df

In [47]:
#実行の準備
full_merge_df_fin["state"] = train['state']
agg_df = agg_func_duration(full_merge_df_fin, group_cols[0])

In [48]:
#関数の実行
for c in group_cols[1:len(group_cols)]:
  agg_df = pd.merge(agg_df, agg_func_duration(full_merge_df_fin, c), left_index=True, right_index=True)

for c in group_cols:
  agg_df = pd.merge(agg_df, agg_func_state(full_merge_df_fin, c), left_index=True, right_index=True)

In [49]:
agg_df

,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y
0,-0.434088,-0.220204,1.351356,-2.536187,0.618778,-0.019053,-0.220204,4.742618,-2.618901,0.983125,-0.090545,-0.220204,4.659904,-2.618901,1.047584,-0.111216,-0.220204,2.261207,-2.618901,0.911437,-0.027740,-0.220204,4.742618,-2.618901,1.007448,-0.438689,-0.220204,1.020501,-2.618901,0.595566,1.954413,2.178493,4.742618,0.606933,0.575559,-0.434088,-0.220204,1.351356,-2.536187,0.618778,0.258354,-0.220204,2.261207,-0.964627,0.889830,-0.019053,-0.220204,4.742618,-2.618901,0.983125,-0.090545,-0.220204,4.659904,-2.618901,1.047584,-0.111216,-0.220204,2.261207,-2.618901,0.911437,-0.027740,-0.220204,4.742618,-2.618901,1.007448,-0.438689,-0.220204,1.020501,-2.618901,0.595566,1.954413,2.178493,4.742618,0.606933,0.575559,-0.434088,-0.220204,1.351356,-2.536187,0.618778
1,2.073298,2.261207,4.742618,0.689647,0.464943,-0.019053,-0.220204,4.742618,-2.618901,0.983125,0.107058,-0.220204,2.261207,-2.618901,1.016086,0.176685,-0.220204,2.261207,-2.618901,1.052416,-0.003448,-0.220204,4.742618,-2.618901,1.040187,2.004836,2.261207,4.742618,0.606933,0.535532,1.954413,2.178493,4.742618,0.606933,0.575559,2.073298,2.261207,4.742618,0.689647,0.464943,0.242664,-0.220204,4.329049,-2.618901,1.028299,-0.019053,-0.220204,4.742618,-2.618901,0.983125,0.107058,-0.220204,2.261207,-2.618901,1.016086,0.176685,-0.220204,2.261207,-2.618901,1.052416,-0.003448,-0.220204,4.742618,-2.618901,1.040187,2.004836,2.261207,4.742618,0.606933,0.535532,1.954413,2.178493,4.742618,0.606933,0.575559,2.073298,2.261207,4.742618,0.689647,0.464943
2,0.458630,-0.137490,3.915481,-1.957192,0.997559,-0.019053,-0.220204,4.742618,-2.618901,0.983125,-0.090545,-0.220204,4.659904,-2.618901,1.047584,0.102029,-0.220204,4.659904,-2.536187,1.200040,-0.027740,-0.220204,4.742618,-2.618901,1.007448,-0.076033,-0.220204,3.915481,-2.536187,0.865704,-0.326178,-0.220204,1.351356,-2.618901,0.565076,0.458630,-0.137490,3.915481,-1.957192,0.997559,-0.074289,-0.220204,4.659904,-2.536187,0.931173,-0.019053,-0.220204,4.742618,-2.618901,0.983125,-0.090545,-0.220204,4.659904,-2.618901,1.047584,0.102029,-0.220204,4.659904,-2.536187,1.200040,-0.027740,-0.220204,4.742618,-2.618901,1.007448,-0.076

In [52]:
#データのマージ
full_merge_df_fin = pd.merge(full_merge_df_fin, agg_df, left_index=True, right_index=True)
full_merge_df_fin.drop(['state'], axis=1, inplace=True)

# データセットの整理

In [54]:
train_df = pd.DataFrame(full_merge_df_fin.loc[:len(train),])
test_df = pd.DataFrame(full_merge_df_fin.loc[len(train):19591,])

In [55]:
test_df = test_df.reset_index()
test_df.drop(['index'], axis=1, inplace=True)
test_df.head()

,goal,country,duration,category1,category2,goal_1-1000,goal_100000+,goal_10001-11000,goal_1001-2000,goal_11001-12000,goal_12001-13000,goal_13001-14000,goal_14001-15000,goal_15001-16000,goal_16001-17000,goal_17001-18000,goal_18001-19000,goal_19001-20000,goal_20001-21000,goal_2001-3000,goal_21001-22000,goal_22001-23000,goal_23001-24000,goal_24001-25000,goal_25001-26000,goal_26001-27000,goal_27001-28000,goal_28001-29000,goal_29001-30000,goal_30001-31000,goal_3001-4000,goal_31001-32000,goal_32001-33000,goal_33001-34000,goal_34001-35000,goal_35001-36000,goal_36001-37000,goal_37001-38000,goal_38001-39000,goal_39001-40000,goal_40001-41000,goal_4001-5000,goal_41001-42000,goal_42001-43000,goal_43001-44000,goal_44001-45000,goal_45001-46000,goal_46001-47000,goal_47001-48000,goal_48001-49000,goal_49001-50000,goal_50001-51000,goal_5001-6000,goal_51001-52000,goal_52001-53000,goal_53001-54000,goal_54001-55000,goal_55001-56000,goal_56001-57000,goal_57001-58000,goal_58001-59000,goal_59001-60000,goal_60001-61000,goal_6001-7000,goal_61001-62000,goal_62001-63000,goal_63001-64000,goal_64001-65000,goal_65001-66000,goal_66001-67000,goal_67001-68000,goal_68001-69000,goal_69001-70000,goal_70001-71000,goal_7001-8000,goal_71001-72000,goal_72001-73000,goal_73001-74000,goal_74001-75000,goal_75001-76000,goal_76001-77000,goal_77001-78000,goal_78001-79000,goal_79001-80000,goal_80001-81000,goal_8001-9000,goal_81001-82000,goal_82001-83000,goal_83001-84000,goal_84001-85000,goal_85001-86000,goal_86001-87000,goal_87001-88000,goal_88001-89000,goal_89001-90000,goal_90001-91000,goal_9001-10000,goal_91001-92000,goal_92001-93000,goal_93001-94000,goal_94001-95000,goal_95001-96000,goal_96001-97000,goal_97001-98000,goal_98001-99000,goal_99001-100000,country_AT,country_AU,country_BE,country_CA,country_CH,country_DE,country_DK,country_ES,country_FR,country_GB,country_HK,country_IE,country_IT,country_JP,country_LU,country_MX,country_NL,country_NO,country_NZ,country_SE,country_SG,country_US,category1_art,category1_comics,category1_crafts,category1_dance,category1_design,category1_fashion,category1_film & video,category1_food,category1_games,category1_journalism,category1_music,category1_photography,category1_publishing,category1_technology,category1_theater,category2_3d printing,category2_academic,category2_accessories,category2_action,category2_animals,category2_animation,category2_anthologies,category2_apparel,category2_apps,category2_architecture,category2_art books,category2_audio,category2_bacon,category2_blues,category2_calendars,category2_camera equipment,category2_candles,category2_ceramics,category2_children's books,category2_childrenswear,category2_chiptune,category2_civic design,category2_classical music,category2_comedy,category2_comic books,category2_community gardens,category2_conceptual art,category2_cookbooks,category2_country & folk,category2_couture,category2_crochet,category2_digital art,category2_diy,category2_diy electronics,category2_documentary,category2_drama,category2_drinks,category2_electronic music,category2_embroidery,category2_events,category2_experimental,category2_fabrication tools,category2_faith,category2_family,category2_fantasy,category2_farmer's markets,category2_farms,category2_festivals,category2_fiction,category2_fine art,category2_flight,category2_food trucks,category2_footwear,category2_gadgets,category2_gaming hardware,category2_glass,category2_graphic design,category2_graphic novels,category2_hardware,category2_hip-hop,category2_horror,category2_illustration,category2_immersive,category2_indie rock,category2_installations,category2_interactive design,category2_jazz,category2_jewelry,category2_kids,category2_knitting,category2_latin,category2_letterpress,category2_literary journals,category2_literary spaces,category2_live games,category2_makerspaces,category2_metal,category2_mixed media,category2_mobile games,category2_movie theaters,category2_music videos,category2_musical,category2_narrative film,category2_nature,category2_nonfict

# lgbによる予測

In [56]:
target = train['state']

In [57]:
# 3分割交差検証を指定し、インスタンス化
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5)  # 3分割交差検証のためにインスタンス化

# スコアとモデルを格納するリスト
score_list = []
models = []

for fold_, (train_index, valid_index) in enumerate(kf.split(train, target)):
    train_x = train_df.iloc[train_index]
    valid_x = train_df.iloc[valid_index]
    train_y = target[train_index]
    valid_y = target[valid_index]
    
    print(f'fold{fold_ + 1} start')

    gbm = lgb.LGBMClassifier(objective='binary',num_boost_round=50000)
    gbm.fit(train_x, train_y, eval_set = [(valid_x, valid_y)],
                early_stopping_rounds=100,
                verbose= 100) # 学習の状況を表示しない
    
    oof = gbm.predict(valid_x, num_iteration=gbm.best_iteration_)
    score_list.append(round(accuracy_score(valid_y, oof)*100,2))
    models.append(gbm)  # 学習が終わったモデルをリストに入れておく
    print(f'fold{fold_ + 1} end\n' )
print(score_list, '平均score', np.mean(score_list), "%")  

fold1 start
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.431378
[200]	valid_0's binary_logloss: 0.438838
Early stopping, best iteration is:
[125]	valid_0's binary_logloss: 0.429801
fold1 end

fold2 start
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.425726
Early stopping, best iteration is:
[86]	valid_0's binary_logloss: 0.424399
fold2 end

fold3 start
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.428933
Early stopping, best iteration is:
[81]	valid_0's binary_logloss: 0.426923
fold3 end

fold4 start
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.398988
[200]	valid_0's binary_logloss: 0.399813
Early stopping, best iteration is:
[112]	valid_0's binary_logloss: 0.396125
fold4 end

fold5 start
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0

In [58]:
# testの予測
test_pred = np.zeros((len(test), 5))  # 行:len(test), 列:3のall zeroの配列を用意

for fold_, gbm in enumerate(models):  # 学習ずみのmodelをgbmに入れる
    pred_ = gbm.predict(test_df, num_iteration=gbm.best_iteration_)  # testの予測
    test_pred[:, fold_] = pred_  # １回目は0列目、2回目は1列目、2回目は3列目に格納

pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)  # 平均をとって、0と１に変換

# submission

In [59]:
sub = pd.DataFrame(test['id'])

In [60]:
sub['state'] = pred
sub.head()

,id,state
0,test_00000,1
1,test_00001,1
2,test_00002,1
3,test_00003,0
4,test_00004,0


In [61]:
sub.to_csv('sub/06_lgb2.csv',index=False, header=None)